In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from stable_baselines3 import DDPG
from deep_learning.networks.lstm import lstm_regressor
import torch

In [2]:
data = pd.read_csv('../data/data_all_radians.csv')
data['joystickX'] = data['joystickX'] * -1
proficiencies = pd.read_csv('../data/overall_proficiency.csv')

In [5]:
data.peopleTrialKey.unique()

array(['1_co_P13/12_600back_Block3_trial_012.csv',
       '1_co_P13/20_600back_Block5_trial_020.csv',
       '1_co_P13/06_600back_Block2_trial_006.csv', ...,
       '2_xz_P11/07_600back_Block2_trial_007.csv',
       '2_xz_P11/09_600back_Block3_trial_009.csv',
       '2_xz_P11/04_600back_Block1_trial_004.csv'], dtype=object)

In [56]:
trial_data = data[data.peopleTrialKey == "2_lp_P4/20_600back_Block5_trial_020.csv"]

trial_data

,seconds,trialPhase,currentPosRollRadians,currentVelRollRadians,joystickX,peopleName,peopleTrialKey
8792537,0.033,1,-0.003068,0.000000,0.008057,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8792538,0.041,1,-0.003068,0.000843,0.008057,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8792539,0.050,1,-0.003068,0.004400,0.008057,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8792540,0.060,1,-0.003068,0.008146,0.008057,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8792541,0.068,1,-0.003068,0.008146,0.008057,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
...,...,...,...,...,...,...,...
8797326,100.276,3,0.002684,-0.008988,-0.000031,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8797327,100.298,3,0.002541,-0.008988,-0.000031,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8797328,100.320,3,0.002397,-0.008988,-0.000031,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv
8797329,100.341,3,0.002253,-0.008988,-0.000031,2_ms_P2/,2_ms_P2/20_600back_Block5_trial_020.csv


In [7]:
rl_path = '../working_models/assistants/ddpg_penalizeDeflection.zip'
dl_path = '../working_models/pilots/pilot_mars_lstm_good_small_window.ckpt'

In [8]:
rl_model = DDPG.load(rl_path)
rl_model

In [9]:
dl_model = lstm_regressor.load_from_checkpoint(dl_path, map_location=torch.device('cpu'))
dl_model

lstm_regressor(
  (lstm): LSTM(3, 100, num_layers=4, batch_first=True, dropout=0.1)
  (hidden_layers): ModuleList(
    (0-2): 3 x Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
      (2): Dropout(p=0.1, inplace=False)
    )
  )
  (output_layer): Linear(in_features=100, out_features=1, bias=True)
  (criterion): HuberLoss()
)

In [10]:
print(trial_data['seconds'].values[1900:2100][0], trial_data['seconds'].values[1900:2100][-1])

39.572 43.775


In [11]:
from deep_learning.utils.mars_dataset import MarsDataset

In [12]:
# trial_data = trial_data.iloc[1850:2150, :]

In [57]:
# test_ppt_trial = "2_ms_P2/20_600back_Block5_trial_020.csv"

past_window_size = 10
future_time = 1
lsmt_format = True

test_ppt_dataset = MarsDataset(trial_data,    
                                    past_window_size=past_window_size, 
                                    future_time=future_time, lstm_format=lsmt_format, 
                                    Xmeans=dl_model.Xmeans.reshape(1, -1), Xstds=dl_model.Xstds.reshape(1, -1), Tmeans=dl_model.Tmeans.reshape(1, -1), Tstds=dl_model.Tstds.reshape(1, -1))


features = test_ppt_dataset.features_physics.float()
print(features.shape)
Xs = features.reshape(-1,past_window_size+1, 3)
dl_preds = dl_model.use(Xs, )
dl_seconds = np.float32(test_ppt_dataset.metas[:,0])
dl_actual = test_ppt_dataset.targets.numpy()

# plot_predicted_joystick_with_actual_trial(joystick_model, test_ppt_dataset, training_parameters, device=device)

100%|██████████| 1/1 [00:00<00:00, 25.09it/s]


torch.Size([4757, 33])


In [63]:
from deep_learning.utils.utils import get_observations_actions_arrays


test_observations, rl_actual, rl_seconds = get_observations_actions_arrays(trial_data, proficiencies, {'proficiency': ['Medium']}, train=False,train_fraction=0)

rl_preds, _ = rl_model.predict(torch.tensor(test_observations, device='cpu', dtype=torch.float32))

In [67]:
# plottting rl model predictions vs dl model predictions

# from python_vip.plotting import plot_trial_normal
dl_trial_data = pd.DataFrame({'seconds': dl_seconds, 'actual': dl_actual.reshape(-1), 'predicted': dl_preds.reshape(-1)}, columns=['seconds', 'actual', 'predicted'])
rl_trial_data = pd.DataFrame({'seconds': rl_seconds.reshape(-1), 'actual': rl_actual.reshape(-1), 'predicted': rl_preds.reshape(-1)}, columns=['seconds', 'actual', 'predicted'])


In [68]:
rl_trial_data

,seconds,actual,predicted
0,0.368,0.008057,-0.884941
1,0.388,0.008057,-0.885447
2,0.410,0.008057,-0.884863
3,0.432,0.008057,-0.885152
4,0.452,0.008057,-0.885269
...,...,...,...
4753,100.276,-0.000031,-0.885047
4754,100.298,-0.000031,-0.884909
4755,100.320,-0.000031,-0.884771
4756,100.341,-0.000031,-0.884632


In [71]:
start= 0
end = 30

dl_trial_data = dl_trial_data[(dl_trial_data.seconds >= start) & (dl_trial_data.seconds <= end)]
rl_trial_data = rl_trial_data[(rl_trial_data.seconds >= start) & (rl_trial_data.seconds <= end)]
trial_data = trial_data[(trial_data.seconds >= start) & (trial_data.seconds <= end)]

In [72]:
len(trial_data), len(rl_trial_data)

(1494, 1494)

In [73]:
np.sum(rl_trial_data['predicted'].values > 1)

0

In [ ]:


# start = 1900
# end = 2100

# seconds = trial_data['seconds'].values
# positions = np.degrees(trial_data['currentPosRollRadians'].values)
# # print(max(positions))
# velocities = np.degrees(trial_data['currentVelRollRadians'].values)
# joysticks = trial_data['joystickX'].values



plt.rcParams.update({'font.size': 32})
fig, ax = plt.subplots(figsize=(20,10))
fig.subplots_adjust(right=0.825, left=0.1)

twin1 = ax.twinx()
# twin2 = ax.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
# twin2.spines.right.set_position(("outward", 127))

p1, = twin1.plot(trial_data['seconds'], np.degrees(trial_data['currentPosRollRadians'].values), "-x", color='black', label="Position (degrees)", zorder=100, markersize=8)

p2, = ax.plot(trial_data['seconds'], trial_data['joystickX'].values, "o", color='red', label="Actual", zorder=200, markersize=12)
p2, = ax.plot(rl_trial_data['seconds'], rl_trial_data['predicted'].values, "--", color='blue', label="RL", zorder=300, markersize=12, linewidth=2.2)
p2, = ax.plot(dl_trial_data['seconds'], dl_trial_data['predicted'].values, "-x", color='green', label="DL", zorder=400, markersize=12, linewidth=2.2)
# p3, = twin2.plot(dl_seconds, dl_preds, "b--", label="DL Joystick Deflections", zorder=300)


positions_max = 70
actions_max = 1.3
# velocities_max = np.max(np.absolute(velocities)) + 5
# ax.set_xlim(0, 2)
twin1.set_ylim(-positions_max, positions_max)
ax.set_ylim(-actions_max, actions_max)
# twin2.set_ylim(-actions_max, actions_max)

ax.set_xlabel("Time (s)")
twin1.set_ylabel("Position (degrees)")
ax.set_ylabel("Joystick Deflections")
# twin2.set_ylabel("DL Joystick Deflections")

twin1.yaxis.label.set_color(p1.get_color())
# twin1.yaxis.label.set_color(p2.get_color())
# twin2.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)

twin1.tick_params(axis='y', colors=p1.get_color(), **tkw)
# twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
# twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

ax.legend()
plt.tight_layout()

# plt.axhline(y=60, color='blue', linestyle='dashed')
# plt.axhline(y=-60, color='blue', linestyle='dashed')

plt.savefig('output/ms_P2_trial_2_20_plot_predicted_deflections_pilot_mars_lstm_good_small_window_ddpg_penalizeDeflection.png', dpi=300)

In [ ]:
# from python_vip.plotting import plot_trial_normal

# start = 1900
# end = 2100
# trial_data = data[data.peopleTrialKey == "1_co_P13/12_600back_Block3_trial_012.csv"]
seconds = trial_data['seconds'].values
print(seconds[0], seconds[-1])
positions = np.degrees(trial_data['currentPosRollRadians'].values)
print(max(positions))
velocities = np.degrees(trial_data['currentVelRollRadians'].values)
joysticks = trial_data['joystickX'].values

plt.rcParams.update({'font.size': 32})
fig, ax = plt.subplots(figsize=(20,10))
fig.subplots_adjust(right=0.825, left=0.1)

twin1 = ax.twinx()
twin2 = ax.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines.right.set_position(("outward", 127))

p1, = ax.plot(seconds, joysticks, "g-", label="Joystick deflections", zorder=100)
p2, = twin1.plot(seconds,velocities, "r--", label="theta_dot(t)/angular velocity", zorder=200)
p3, = twin2.plot(seconds, positions, "b-", label="theta(t)/angular position", zorder=300)


positions_max = 65
actions_max = 1.1
velocities_max = np.max(np.absolute(velocities)) + 5
# ax.set_xlim(0, 2)
ax.set_ylim(-actions_max, actions_max)
twin1.set_ylim(-velocities_max, velocities_max)
twin2.set_ylim(-positions_max, positions_max)

# ax.set_xlabel("Time (s)")
ax.set_ylabel("Joystick Deflection")
twin1.set_ylabel("Velocity (degrees/sec)")
twin2.set_ylabel("Position (degrees)")

ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p2.get_color())
twin2.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

# plt.axhline(y=60, color='blue', linestyle='dashed')
# plt.axhline(y=-60, color='blue', linestyle='dashed')

# plt.savefig('output/co_P13_trial_1_12_plot.png', dpi=300)